<center><b> 1D time-dependent Frantz Nodvik equations </b></center>

<br> du1/dt + gamma*K*u1*u2 = 0           u1 = Delta = # density of excited states 
<br> du2/dt + c*du2/dx - K*u1*u2 = 0      u2 = n_ph  = # density of photons
<br>                                      c = speed of light
<br>                                      K = sigma * c
<br> IC:  TBD...
<br> BC:  TBD...
<br> This code is distributed under the Apache License, Version 2.0, January 2004.
<br> Copyright (c) 2020 RadiaSoft LLC, All rights reserved
<br> Author:  David Bruhwiler                                     
<br> The laser pulse duration $T_{laser}$ is typically 200 ps to 1 ns.
<br> The Ti-Sapphire phase relaxation time $T_{crystal}$ is between $10 fs$ and $10 ps$.
<br> Because $T_{crystal} << T_{laser}$, we can use the Frantz-Nodvik (F-N) rate equations.
<br> This approach can be used for other crystals as well.

In [1]:
# Primary library for the FEniCS Python API (should not import *)
from dolfin import *

In [2]:
class DataFrantzNodvik1D(object):
    """ Hold simulaton data for 1D Frantz-Nodvik simulations

    Attributes:
        n_dx (int): resolution of the 1D mesh along the x-axis
        dt (float): simulation time step
        c_gamma (float): ratio of degeneracies for crystal states []
        c_sigma (float): photon emission crosssection of the crystal [m^2]
        c_width (float): width of the crystal amplifier [m]
        c_density (float): # density of excited states in the crystal [cm^-3]
        lp_density (float): # density of photons in the laser pulse [cm^-3]
        lp_width (float): full width of the laser pulse [s]
    """

    def __init__(self, sim_time, dt, n_dx, c_gamma, c_sigma, c_width, c_density, lp_density, lp_width):
        self.sim_time = sim_time
        self.dt = dt
        self.n_dx = n_dx
        self.c_gamma = c_gamma
        self.c_sigma = c_sigma
        self.c_width = c_width
        self.c_density = c_density
        self.lp_density = lp_density
        self.lp_width = lp_width

        # to be deleted...
        print ('')
        print ('Input parameters --')
        print ('  simulation time [s]      = %g' % (sim_time) )
        print ('  # of mesh cells          = %d' % (n_dx) )
        print ('  time step, dt [s]        = %g' % (dt) )
        print ('  c_gamma [ ]              = %g' % (c_gamma) )
        print ('  c_sigma [m^2]            = %g' % (c_sigma) )
        print ('  crystal width [m]        = %g' % (c_width) )
        print ('  # dens ex. states [m^-3] = %g' % (c_density) )
        print ('  # dens photons [m^-3]    = %g' % (lp_density) )
        print ('  width of laser pulse [s] = %g' % (lp_width) )
        
        import scipy.constants
        self.c = scipy.constants.c         # speed of light [m/s]
        self.K = self.c_sigma * self.c     # [m^3/s]

        # create a 1D mesh on the interval [0.,c_width].
        self.x_left = 0.
        self.x_right = self.c_width
        self.mesh = IntervalMesh(self.n_dx, self.x_left, self.x_right)

        # let's try flat line segments, with no requirement of continuity
        self.p1 = FiniteElement("Discontinuous Lagrange","interval",0)
    
        # define an 'element' appropriate for two 1D functions
        self.element = MixedElement([self.p1, self.p1])
    
        # define the test functions: v=[v_1, v_2]
        self.v = FunctionSpace(self.mesh, self.element)
        self.v_1, self.v_2 = TestFunctions(self.v)

        # define solution functions: u=[u_1, u_2]
        self.u = Function(self.v)
        self.u_1, self.u_2 = split(self.u)

        # solutions from the previous time step: u_n=[u_n1, u_n2]
        self.u_n = Function(self.v)
        self.u_n1, self.u_n2 = split(self.u_n)
        
        return

In [3]:
def frantz_nodvik_1d_solver(sim_time, fn1d):
    """Frantz-Nodvik 1D solver for wavefront amplification in a pumped crystal amplifier

    Args:
        sim_time (float): simulation time [s]
        fn1d (object): instance of the DataFrantzNodvik1D class

    Returns:
        NA
    """
    # to be deleted...
    print ('')
    print ('Simulation time [s] = %g' % (sim_time) )

    # boundary conditions imposed at x=0.
    # assume the incident wavefront has a square profile
#    u_left = Expression('t>=0. && t<=fn1d.lp_width ? fn1d.lp_density : 0.', degree=1, t=0.)
#    def on_left(x, on_boundary):
#        return (on_boundary and near(x[0], fn1d.x_left))
    
    # apply the BCs
#    u_left.t = sim_time
#    bc = DirichletBC(fn1d.V, u_left, on_left)

    dt = Constant(fn1d.dt)

    # define the variational problem via function F
    F = ( (fn1d.u_1 - fn1d.u_n1) * fn1d.v_1 / dt \
        + fn1d.c_gamma * fn1d.K * fn1d.u_1 * fn1d.u_2 * fn1d.v_1 \
        + (fn1d.u_2 - fn1d.u_n2) * fn1d.v_2 / dt \
        + fn1d.c * D(fn1d.u_2, 0) * fn1d.v_2 \
        - fn1d.K * fn1d.u_1 * fn1d.u_2 * fn1d.v_2 \
        ) * dx

    # specify the Jacobian
    J = derivative(F, fn1d.u)

    # solve the system of equations
    solve(F==0, u, bc, J=J)

    return

In [16]:
def frantz_nodvik_1d_setup():
    """Setup choices for Frantz-Nodvik 1D solver

    Args:
        NA

    Returns:
        fn1d (object):  instance of the DataFrantzNodvik1D class
    """
    # specify the desired parameters (edit by hand for now)
    sim_time = 1.          # [s]
    n_steps = 100
    dt = sim_time/n_steps  # [s]
    n_dx = 200
    c_gamma = 1.5          # dimensionless
    c_sigma = 1000.        # [m^2]
    c_width = 0.01         # [m]
    c_density = 1000.      # [m^-3]
    lp_density = 100       # [m^-3]
    lp_width = 1.e-9       # [s]
    
    # create an instance of the data class
    fn1d = DataFrantzNodvik1D(sim_time, dt, n_dx, c_gamma, c_sigma, \
                              c_width, c_density, lp_density, lp_width)
    
    # specify and apply the initial conditions
    fn1d.v_1 = Constant(c_density)  # assuming density of states is constant
    fn1d.v_1 = Constant(0.)         # assuming no photons in crystal at t=0

    # set U and U0 by interpolation.
    fn1d.u_n.assign(fn1d.u)

    return

In [17]:
# initialize the data object
fn1d = frantz_nodvik_1d_setup()


Input parameters --
  simulation time [s]      = 1
  # of mesh cells          = 200
  time step, dt [s]        = 0.01
  c_gamma [ ]              = 1.5
  c_sigma [m^2]            = 1000
  crystal width [m]        = 0.01
  # dens ex. states [m^-3] = 1000
  # dens photons [m^-3]    = 100
  width of laser pulse [s] = 1e-09
